In [19]:
%load_ext autoreload
%autoreload 2

## Standard libraries
import os
import numpy as np
import random
from PIL import Image
from types import SimpleNamespace
from dotenv import load_dotenv

load_dotenv()

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()

## PyTorch
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim

# Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
import torchvision.models as models

import lightning as L
from torch.utils.data import DataLoader

from example_submission import TaskDataset
from torch.utils.data import Dataset
from typing import Tuple

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users\arkad\AppData\Local\Temp\ipykernel_10372\980968177.py:18: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


In [27]:
class ModifedDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.ids = dataset.ids
        self.imgs = dataset.imgs
        self.labels = [int(l) for l in dataset.labels]

        self.transform = transform

        self.number_of_classes = len(set(self.labels))
        self.classes_mapping = {label: i for i, label in enumerate(set(self.labels))}

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]
        if not self.transform is None:
            img = self.transform(img)
        label = self.classes_mapping[self.labels[index]]
        return id_, img, label

    def __len__(self):
        return len(self.ids)

In [28]:
DATASET_PATH = os.getenv("TASK_2_DATA_PUBLIC_PATH")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.shape[0] == 1 else x)
])

dataset = torch.load(DATASET_PATH)
dataset = ModifedDataset(dataset, transform)

In [24]:
len(set(dataset.labels))

50

In [31]:
# Define the split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calculate the lengths for each split
total_size = len(dataset)
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
test_size = total_size - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Create DataLoader objects for each split
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [30]:
for batch in train_loader:
    print(batch)
    break

[tensor([268238]), tensor([[[[0.2196, 0.3216, 0.3529,  ..., 0.0471, 0.0549, 0.0667],
          [0.1451, 0.2353, 0.2902,  ..., 0.0471, 0.0510, 0.0588],
          [0.0941, 0.1451, 0.2314,  ..., 0.0588, 0.0510, 0.0471],
          ...,
          [0.1686, 0.1686, 0.1725,  ..., 0.0588, 0.0745, 0.0863],
          [0.2000, 0.1804, 0.1647,  ..., 0.0549, 0.0627, 0.0667],
          [0.2039, 0.1765, 0.1569,  ..., 0.0510, 0.0510, 0.0588]],

         [[0.2196, 0.3216, 0.3529,  ..., 0.0471, 0.0549, 0.0667],
          [0.1451, 0.2353, 0.2902,  ..., 0.0471, 0.0510, 0.0588],
          [0.0941, 0.1451, 0.2314,  ..., 0.0588, 0.0510, 0.0471],
          ...,
          [0.1686, 0.1686, 0.1725,  ..., 0.0588, 0.0745, 0.0863],
          [0.2000, 0.1804, 0.1647,  ..., 0.0549, 0.0627, 0.0667],
          [0.2039, 0.1765, 0.1569,  ..., 0.0510, 0.0510, 0.0588]],

         [[0.2196, 0.3216, 0.3529,  ..., 0.0471, 0.0549, 0.0667],
          [0.1451, 0.2353, 0.2902,  ..., 0.0471, 0.0510, 0.0588],
          [0.0941, 0.14

In [38]:
class StealingModule(L.LightningModule):
    def __init__(self, model, lr=1e-3):
        super().__init__()
        self.save_hyperparameters()
        
        self.model = model
        self.criterion = nn.CrossEntropyLoss()
        self.lr = lr

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        id, x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("train_acc", acc, prog_bar=True, logger=True)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        id, x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("val_loss", loss, prog_bar=True, logger=True)
        self.log("val_acc", acc, prog_bar=True, logger=True)

    def test_step(self, batch, batch_idx):
        id, x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("test_loss", loss)
        self.log("test_acc", acc)

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr)

class Model(nn.Module):
    def __init__(self, model_name, num_classes):
        super().__init__()
        match model_name:
            case "resnet18":
                self.backbone = models.resnet18(pretrained=True)
            case _:
                raise NotImplementedError

        self.representation = nn.Linear(self.backbone.fc.in_features, 1024)
        self.projection = nn.Linear(1024, num_classes)
            
        self.backbone.fc = nn.Identity()
    
    def forward(self, x):
        x = self.backbone(x)
        x = self.representation(x)
        x = self.projection(x)
        return x

In [36]:
model = Model(model_name="resnet18", num_classes=50)
lightning_model = StealingModule(model)
trainer = L.Trainer(max_epochs=10, accelerator="gpu" if torch.cuda.is_available() else "cpu", logger=False)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [37]:
trainer.fit(lightning_model, train_dataloaders=train_loader, val_dataloaders=val_loader)

d:\ProgramFiles\Miniconda3\envs\thesis\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:653: Checkpoint directory d:\Code\ensemble-ai\task_2\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Model            | 11.2 M
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.809    Total estimated model params size (MB)


Epoch 0:  10%|▉         | 4/41 [00:00<00:02, 17.59it/s, train_acc=0.398, train_loss=2.260] 

d:\ProgramFiles\Miniconda3\envs\thesis\Lib\site-packages\lightning\pytorch\core\module.py:507: You called `self.log('train_acc', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`


Epoch 9: 100%|██████████| 41/41 [00:02<00:00, 16.94it/s, train_acc=0.800, train_loss=0.488, val_loss=1.380, val_acc=0.606]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 41/41 [00:02<00:00, 14.10it/s, train_acc=0.800, train_loss=0.488, val_loss=1.380, val_acc=0.606]
